# Ising Formulation of Number Partitioning Problem

## Ising Formulation

$$
H = \left( \sum_{i=1}^{N}n_i s_i \right)^2 = \left( \sum_{s_{i}=1}^{N} n_i - \sum_{s_{i}=-1}^{N} n_i \right)^2 =\sum_{i=1}^{N} \sum_{j=1}^{N} n_i n_j s_i s_j 
$$

$n_i\ (i=1,\dots, N)$ represents the element in the list of numbers to be divided.


(details : https://www.frontiersin.org/articles/10.3389/fphy.2014.00005/full)

## Solving Number Partitioning Problem with D-Wave machine

Install the necessary libraries.

In [48]:
import numpy as np
import random 
import matplotlib.pyplot as plt
import pandas as pd
from dwave.system import DWaveCliqueSampler

Define a function to randomly generate a list of numbers to be partitioned.

In [2]:
def make_number_partitioning_problem(N, number_range):
    J = np.zeros((N, N))
    h = np.zeros(N)
    number_list = [random.choice(number_range) for i in range(N)]  # Randomly generates a list of numbers to be partitioned
    for i in range(N):
        h[i] = number_list[i]**2
        for j in range(N):
            if i < j:
                J[i, j] = number_list[i] * number_list[j]
                J[j, i] = J[i, j]
    return h, J, number_list

Given a spin configuration (the answer to the problem), define a function to compute the Hamiltonian of a number-partitioning problem.

In [44]:
def compute_hamiltonian(s_np, J_np, h_np, N):
    return np.sum([h_np[i] if i==j else J_np[i,j] * s_np[i] * s_np[j] for i in range(N) for j in range(N)])

Set the range of the list of numbers to be divided.

In [16]:
number_range = [1, 2, 3, 4, 5]

Create one problem randomly. In this case, the size of the problem is 10.

In [17]:
N = 10
h_np = np.zeros(N)
J_np = np.zeros((N, N))
number = np.zeros(N)
h_np, J_np, number_list = make_number_partitioning_problem(N=N, number_range=number_range)

Specify the solver to be used and write the API token.

In [5]:
from dwave.system import DWaveCliqueSampler
sampler_config = {"solver": "Advantage_system6.1", "token": "YOUR TOKEN"}
sampler = DWaveCliqueSampler(**sampler_config)

Get the solution from the d-wave machine.

In [18]:
answer = sampler.sample_ising(h_np, J_np, num_reads=100)


The following is a tabular representation of the partitions obtained.

In [40]:
table_list = [number_list, [answer.first.sample[i] for i in range(N)]]
df = pd.DataFrame(table_list)
df.index = ["number", "Group"]
df

,0,1,2,3,4,5,6,7,8,9
number,5,5,2,4,5,1,1,5,1,4
Group,-1,1,1,1,-1,1,1,-1,1,-1


Calculate the sum for a group with a spin value of +1 ( index : 0, 4, 7, 9 ).
$$5+5+5+4 = 19$$
Next, calculate the sum for a group with a spin value of -1 ( index : 1, 2, 3, 5, 6, 8 ).
$$5+2+4+1+1+1 = 14$$

Thus, the difference between the group values is$$19-14=5$$

The square of this value ($5^2=25$) corresponds to the value of the Hamiltonian $H$.

In [47]:
hamiltonian_np = compute_hamiltonian(answer.first.sample, J_np, h_np, N)
print(hamiltonian_np)

25.0
